In [14]:
import pickle

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer

import funciones_auxiliares_hito_2 as aux

sns.set_style()
plt.rcParams["figure.figsize"] = (20, 10)

# Modelacion

In [7]:
df = pd.read_csv("input/training_tweets.csv").drop(columns="Unnamed: 0")
df["sentiment"] = aux.codificar_sentimientos(df["sentiment"])
df["procesados_lema"] = df.content.str.lower().apply(aux.preprocesar_texto_lema)
df_lema = aux.obtener_100_palabras_mas_frecuentes(df["procesados_lema"])


## Optimizacion

- La optimizacion consistira en filtrar el numero de palabras segun su frecuencia. Esto permitira
reducir la complejidad computacional, esperando tener un desempeno del modelo similar a los modelos
sin optimizacion.

In [9]:
largo_df = df.shape[0]
filtro_frecuencias = int(round(largo_df * 0.0004, 0))

In [10]:
palabras_a_filtrar = df_lema.query("conteo < @filtro_frecuencias")
palabras_restantes = df_lema.query("conteo >= @filtro_frecuencias")
cambio_en_palabras = df_lema.shape[0] - palabras_a_filtrar.shape[0]
LISTA_PALABRAS_A_DEJAR = set(palabras_restantes.palabra.to_list())

print(f"Las palabras a filtrar son:\n{palabras_a_filtrar}\n")
print(f"Se pasara de {df_lema.shape[0]} a {cambio_en_palabras} palabras")


Las palabras a filtrar son:
            palabra  conteo
2006             ½ï      11
2007             dh      11
2008             pa      11
2009           melt      11
2010         centre      11
...             ...     ...
37676    hysterical       1
37677     hypocrite       1
37678  hypnoticzexy       1
37679      hypnosis       1
37680     lilmickee       1

[35675 rows x 2 columns]

Se pasara de 37681 a 2006 palabras


- Por lo tanto, se puede ver una disminucion considerable en la cantidad de palabras a analizar
en cada modelo predictivo.

- El filtro de palabras se aplicara en el objeto CountVectorizer con el parametro vocabulary.

In [15]:
X = CountVectorizer(stop_words="english", vocabulary=LISTA_PALABRAS_A_DEJAR).fit_transform(
    df["procesados_lema"]
)
y = aux.codificar_sentimientos(df["sentiment"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.335, random_state=1)


# Resumen

En la siguiente tabla se puede observar la diferencia en desempeno entre el modelo con las palabras
filtradas por frecuencia y el que carece de tal filtro:

|Metrica|Modelo Con Filtro|Modelo Sin Filtro|
|-------|-----------------|-----------------|
Mejor Accuracy en Entrenamiento|0.67|0.67|
Mejor Accuracy en Validacion|0.67|0.67|
ROC en Validacion|0.67|0.67|

Se observa que tanto el modelo filtrado, como el modelo sin filtro presentan resultados identicos 
(metricas redondeadas a 2 cifras decimales). Esta tendencia se observa tanto en el conjunto de datos
de entrenamiento, como en el conjunto de datos de validacion.

Por lo tanto, **se utilizara el modelo con el filtro de palabras, ya que reduce considerablemente
el costo computacional, y es mucho mas parsimonioso**.

# Importacion de Modelos Entrenados

Debido a que se entrenaron todos los modelos en el Hito 2, se procedera a importar cada uno de los
modelos ya entrenados.

In [5]:
archivos_a_leer = [
    "modelo_logistic.pickle",
    "modelo_naive_bayes.pickle",
    "modelo_arbol.pickle",
    "modelo_random_forest.pickle",
    "modelo_gradient_boosting.pickle",
]

modelos = {}
for nombre_archivo in archivos_a_leer:
    with open(nombre_archivo, "rb") as file:
        modelo = pickle.load(file)
        modelos[nombre_archivo.split(".")[0]] = modelo


# Reporte comparativo de desempenos

En el Hito 2 se reportaron los desempenos de cada uno de los modelos ya entrenados. Esto se realizo
tanto para el conjunto de entrenamiento y el conjunto de validacion. En la siguiente tabla se
recopila a modo de resumen los resultados.

|Modelo|Entrenamiento Accuracy|Validacion Accuracy|Validacion ROC|
|------|----------------------|-------------------|--------------|
|Logistico|0.67|0.68|0.68|
|Naive Bayes|0.67|0.68|0.68|
|Arbol de clasificacion|0.57|0.57|0.56|
|Random Forest|0.66|0.66|0.65|
|Gradient Boosting|0.66|0.67|0.66|


- Se puede observar que los mejores modelos de clasificacion (segun el valor ROC) fueron el
modelo de regresion logistica o naive bayes. Ambos modelos presentaron un desempeno identico,
tanto en el conjunto de entrenamiento como en el de validacion. Ademas, fueron los modelos que
tuvieron el menor coste computacional, demorando ~8 y ~2 segundos en entrenar, respectivamente.

- El peor modelo fue el arbol de clasificacion, con un valor de ROC de 0.56. Por lo tanto, clasifica
en 6% mejor que un clasificador aleatorio.

- El modelo de Random Forest y Gradient Boosting fueron los modelos mas costosos computacionalmente,
demorando ~3 y ~9 minutos en entrenar, respectivamente. A pesar de su alto costo computacional,
estos modelos tuvieron un resultado comparable (levemente peor) al modelo logistico o naive bayes.

- El valor ROC promedio de los 5 modelos fue de 0.65.

# Exportacion de Mejores Modelos

Teniendo en cuenta los resultados anteriores, los mejores modelos son el modelo de regresion
logistica y naive bayes. Por lo tanto, ambos seran exportados para validacion externa.

In [11]:
modelos_a_exportar = {
    "modelo_1_modelo_logistico.pickle": modelos["modelo_logistic"],
    "modelo_2_modelo_naive_bayes.pickle": modelos["modelo_naive_bayes"],
}

In [13]:
for archivo, modelo in modelos_a_exportar.items():
    with open(archivo, "wb") as file:
        pickle.dump(modelo, file)